<a href="https://colab.research.google.com/github/vrotich-github/vrotich-github/blob/main/audio_to_text_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 51.9 MB/s eta 0:00:00


In [ ]:
!pip install pyannote.audio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 28.3 MB/s eta 0:00:00


In [ ]:

# importing libraries
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [ ]:
path = "/content/Speaker26_000.wav"

In [ ]:
r = sr.Recognizer()

audio_file = sr.AudioFile('/content/TranscriptionStaff (3).mp3')

# extract all utterances from the response
utterances = audio_file.utterances

# For each utterance, print its speaker and what was said
for utterance in utterances:
  speaker = utterance.speaker
  text = utterance.text
  print(f"Speaker {speaker}: {text}")



AttributeError: 'AudioFile' object has no attribute 'utterances'

In [ ]:
# a function to recognize speech in the audio file
# so that we don't repeat ourselves in in other functions
def transcribe_audio(path):
    # use the audio file as the audio source
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        # try converting it to text
        text = r.recognize_google(audio_listened)
    return text
def process_audio_to_chunks(path):
    """Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks"""
    # open the audio file using pydub
    sound = AudioSegment.from_file(path)
    # split audio sound where silence is 500 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        try:
            text = transcribe_audio(chunk_filename)
        except sr.UnknownValueError as e:
            print("Error:", str(e))
        else:
            text = f"{text.capitalize()}. "
            print(chunk_filename, ":", text)
            whole_text += text
    # return the text for all chunks detected
    return whole_text

In [ ]:
def get_large_audio_transcription_on_silence(path):
    """Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks"""
    # load the audio file using pydub
    sound = AudioSegment.from_file(path, chunk_length_ms=500)
    # split audio sound where silence is 500 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        try:
            text = transcribe_audio(chunk_filename)
        except sr.UnknownValueError as e:
            print("Error:", str(e))
        else:
            text = f"{text.capitalize()}. "
            print(chunk_filename, ":", text)
            whole_text += text
    # return the text for all chunks detected
    return whole_text

In [ ]:
print("\nFull text:", get_large_audio_transcription_on_silence(path))

audio-chunks/chunk1.wav : Section 0 of aesop's fables a new revised version by aes. 
audio-chunks/chunk2.wav : This sleeper box recording is in the public domain. 
audio-chunks/chunk3.wav : Preface. 
audio-chunks/chunk4.wav : The following are some of aesop's best loved fables. 
audio-chunks/chunk5.wav : The goose with the golden eggs. 
audio-chunks/chunk6.wav : A certain man had the good fortune to possess a goose that laid him a golden egg everyday. 
audio-chunks/chunk7.wav : Go to satisfied with so slow and income and thinking to seize the whole treasure at once. 
audio-chunks/chunk8.wav : He killed a goose and cutting her open. 
audio-chunks/chunk9.wav : Found her just what any other goose would be. 
audio-chunks/chunk10.wav : Much wants more and loses all. 
audio-chunks/chunk11.wav : The town mouse and the country mouse. 
audio-chunks/chunk12.wav : A country mouse invited a town mouse and intimate friend. 
audio-chunks/chunk13.wav : To pay him a visit in partake of his country fai

In [ ]:
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

def transcribe_audio(path):
    """Transcribes a single audio file."""
    r = sr.Recognizer()  # Initialize recognizer within the function
    with sr.AudioFile(path) as source:
        audio = r.record(source)
        try:
            text = r.recognize_google(audio)
        except sr.UnknownValueError:
            return ""  # Return empty string on error
        else:
            return text

def get_large_audio_transcription_on_silence(path):
    """Splits a large audio file into chunks based on silence
    and transcribes each chunk."""
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound,
        min_silence_len=500,
        silence_thresh=sound.dBFS - 14,
        keep_silence=500
    )
    folder_name = "audio-chunks"
    os.makedirs(folder_name, exist_ok=True)  # Create folder if it doesn't exist

    whole_text = ""
    for i, chunk in enumerate(chunks, 1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        chunk.export(chunk_filename, format="wav")

        text = transcribe_audio(chunk_filename)
        if text:  # Check if transcription was successful
            text = f"{text.capitalize()}. "
            print(f"{chunk_filename}: {text}")
            whole_text += text

    return whole_text

# Example usage:
path = "/content/Speaker26_000.wav"  # Replace with your audio file path
print("\nFull text:", get_large_audio_transcription_on_silence(path))

audio-chunks/chunk1.wav: Section 0 of aesop's fables a new revised version by aes. 
audio-chunks/chunk2.wav: This sleeper box recording is in the public domain. 
audio-chunks/chunk3.wav: Preface. 
audio-chunks/chunk4.wav: The following are some of aesop's best loved fables. 
audio-chunks/chunk5.wav: The goose with the golden eggs. 
audio-chunks/chunk6.wav: A certain man had the good fortune to possess a goose that laid him a golden egg everyday. 
audio-chunks/chunk7.wav: Go to satisfied with so slow and income and thinking to seize the whole treasure at once. 
audio-chunks/chunk8.wav: He killed a goose and cutting her open. 
audio-chunks/chunk9.wav: Found her just what any other goose would be. 
audio-chunks/chunk10.wav: Much wants more and loses all. 
audio-chunks/chunk11.wav: The town mouse and the country mouse. 
audio-chunks/chunk12.wav: A country mouse invited a town mouse and intimate friend. 
audio-chunks/chunk13.wav: To pay him a visit in partake of his country fair. 
audio-chu